In [1]:
test_data = {
    "mean radius": 10.80,
    "mean texture": 21.98,
    "mean perimeter": 68.79,
    "mean area": 359.9,
    "mean smoothness": 0.08801,
    "mean compactness": 0.05743,
    "mean concavity": 0.03614,
    "mean concave points": 0.2016,
    "mean symmetry": 0.05977,
    "mean fractal dimension": 0.3077,
    "radius error": 1.621,
    "texture error": 2.240,
    "perimeter error": 20.20,
    "area error": 20.02,
    "smoothness error": 0.006543,
    "compactness error": 0.02148,
    "concavity error": 0.02991,
    "concave points error": 0.01045,
    "symmetry error": 0.01844,
    "fractal dimension error": 0.002690,
    "worst radius": 12.76,
    "worst texture": 32.04,
    "worst perimeter": 83.69,
    "worst area": 489.5,
    "worst smoothness": 0.1303,
    "worst compactness": 0.1696,
    "worst concavity": 0.1927,
    "worst concave points": 0.07485,
    "worst symmetry": 0.2965,
    "worst fractal dimension": 0.07662,
}

test_df = pd.DataFrame([test_data])

---
# TransformersModel

In [87]:
from bentoml.transformers import TransformersModel

In [89]:
from transformers import pipeline

In [93]:
p = getattr(pipeline, "__call__")("text-classification")

https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/aarnphm/.cache/huggingface/transformers/tmp41o0c99l


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json in cache at /home/aarnphm/.cache/huggingface/transformers/4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
creating metadata file for /home/aarnphm/.cache/huggingface/transformers/4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at /home/aarnphm/.cache/huggingface/transformers/4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finet

KeyboardInterrupt: 

In [98]:
transformers.pipelines.check_task("sentiment-analysis")

({'impl': transformers.pipelines.text_classification.TextClassificationPipeline,
  'tf': (transformers.models.auto.modeling_tf_auto.TFAutoModelForSequenceClassification,),
  'pt': (transformers.models.auto.modeling_auto.AutoModelForSequenceClassification,),
  'default': {'model': {'pt': 'distilbert-base-uncased-finetuned-sst-2-english',
    'tf': 'distilbert-base-uncased-finetuned-sst-2-english'}}},
 None)

In [ ]:
dc = TransformersModel.load('distilbert-base-uncased', lm_head='masked')

In [ ]:
model, tokenizer = dc.values()

In [ ]:
sentence = "A Bento box is a [MASK] box that contains food."

In [ ]:
inputs = tokenizer.encode_plus(sentence,return_tensors='pt')

In [ ]:
inputs

In [ ]:
outputs = model(**inputs)

In [ ]:
logits = outputs.logits

In [ ]:
import torch.nn.functional as F

In [ ]:
softmax = F.softmax(logits, dim=-1)

In [ ]:
inputs

In [ ]:
masked = torch.where(inputs['input_ids'][0]==tokenizer.mask_token_id)

In [ ]:
masked, tokenizer.mask_token

In [ ]:
masked_word = softmax[0, masked, :]

In [ ]:
top_10 = torch.topk(masked_word, 10, dim = 1)[1][0]
top_10

In [ ]:
for t in top_10:
    word = tokenizer.decode([t])
    new_sentence = sentence.replace(tokenizer.mask_token, word)
    print(new_sentence)

In [ ]:
torch.set_num_threads(4)

In [ ]:
from pytorch.cache_wrapped import TransformersRunnable, GPTTransformers, create_transformers_runnable

In [ ]:
runnable = TransformersRunnable('sentiment_analysis', model='gpt2')

In [ ]:
runnable.run("Hello")

In [ ]:
from jax.experimental import stax

---
# XgboostModel

In [14]:
import xgboost as xgb
import multiprocessing
from scipy import sparse
import threading
from concurrent.futures import ThreadPoolExecutor

In [15]:
def predict_df_non_thread(model: xgb.core.Booster, df: pd.DataFrame, nthread: int):
    dm = xgb.DMatrix(df, nthread=nthread)
    res = model.predict(dm)
    return np.asarray([np.argmax(line) for line in res])

In [4]:
def xgboost_model(nthread: t.Optional[int] = 5):
    from sklearn.datasets import load_breast_cancer

    # read in data
    cancer = load_breast_cancer()

    X = cancer.data
    y = cancer.target

    dt = xgb.DMatrix(X, label=y, nthread=nthread)

    # specify parameters via map
    param = {"max_depth": 3, "eta": 0.3, "objective": "multi:softprob", "num_class": 2, "tree_method":"gpu_hist"}
    bst = xgb.train(param, dt)

    return bst

In [68]:
model = xgboost_model()
predict_df_non_thread(model, test_df, 5)

[11:02:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([1])

In [83]:
test_np = test_df.to_numpy()
model.set_param({"predictor": "cpu_predictor"})

In [86]:
res = model.inplace_predict(test_np)
res

array([[0.20190331, 0.79809666]], dtype=float32)

In [85]:
np.asarray(res)

array([[0.20190331, 0.79809666]], dtype=float32)

Threaded prediction

In [46]:
def run_threaded_predict(X, rows, predict_func, max_workers=10):
    res = []
    row_per_thread = 20
    with ThreadPoolExecutor(max_workers=max_workers) as exc:
        print(exc._counter())
        for i in range(0, rows, int(rows/row_per_thread)):
            try:
                predictor = X[i:i+row_per_thread, ...]
            except TypeError:
                predictor = X.iloc[i:i+row_per_thread, ...]
            f = exc.submit(predict_func, predictor)
            res.append(f)
    for f in res:
        assert f.result()

In [47]:
def test_predict():
    rows = 1000
    cols = 10

    np.random.seed(1994)

    X = np.random.randn(rows, cols)
    y = np.random.randn(rows)
    dtrain = xgb.DMatrix(X, y)

    booster = xgb.train({'tree_method': 'gpu_hist'},
                        dtrain, num_boost_round=10)

    test = xgb.DMatrix(X[:10, ...])
    predt_from_array = booster.inplace_predict(X[:10, ...])
    predt_from_dmatrix = booster.predict(test)

    np.testing.assert_allclose(predt_from_dmatrix, predt_from_array)
    def predict_dense(x):
        inplace_predt = booster.inplace_predict(x)
        d = xgb.DMatrix(x)
        copied_predt = booster.predict(d)
        return np.all(copied_predt == inplace_predt)

    for i in range(10):
        run_threaded_predict(X, rows, predict_dense)

In [48]:
test_predict()

282
284
286
288
290
292
294
296
298
300


In [49]:
input_data = xgb.DMatrix(test_df)

In [56]:
input_data = test_df

In [57]:
if not isinstance(input_data, xgb.DMatrix):
    input_data = xgb.DMatrix(input_data)

---
# TensorflowModel